In [ ]:
import pandas as pd
import warnings
from datetime import datetime
warnings.simplefilter("ignore")

- ## VERIFICAÇÃO HORÍMETRO

In [ ]:
#Entrada da base
Dados1 = pd.read_excel('/content/APONTAMENTOS.xlsx')

Dados1['QTD'] = Dados1['QTD'].replace({',': '.'}, regex=True)
Dados1['NO_HOR_ODOM'] = Dados1['NO_HOR_ODOM'].replace({',': '.'}, regex=True)
Dados1['KM'] = Dados1['KM'].replace({',': '.'}, regex=True)

#Substituir deslocamentos vazios
Dados1['KM'] = Dados1['KM'].fillna(0)
Dados1['QTD'] = Dados1['QTD'].fillna(0)

Dados = Dados1

#Pré-processamento (Filtro 'HR' | Conversão coluna KM e Unidade)
Dados['KM'] = Dados['KM'].astype('float64')
Dados['QTD'] = Dados['QTD'].astype('float64')
Dados['FROTA'] = Dados['FROTA'].astype('int64')
Dados['TIPO_RODADO'] = Dados['TIPO_RODADO'].astype('string')

#Filtrar equipamentos HR
Dados = Dados[Dados['TIPO_RODADO']=='HR']

#Tratamento da coluna data
lista_data1 = []
for k in Dados['HR_OPERACAO']:
  k = str(k)
  convert = k.split()
  parameter = 'j'
  if int(len(convert))<2:
    lista_data1.append((str(convert[0]))+"  00:00:00")
    parameter = 'i'
  if int(len(convert))>=2 and parameter=='j':
    lista_data1.append(" ".join(str(k).split()))

Dados['HR_OPERACAO'] = lista_data1
Dados['HR_OPERACAO']  = pd.to_datetime(Dados['HR_OPERACAO'], format='%d/%m/%Y %H:%M:%S')

#Uso no duplicado
Dados123 = Dados

#Ordenando os valores por FROTA e Horas
Dados = Dados.sort_values(['FROTA', 'HR_OPERACAO']).reset_index(drop=True)

D_1 = list(Dados['NO_HOR_ODOM'])
if len(D_1)>0:
  del D_1[0]
  D_1.append('0')
  Dados['NO_HOR_ODOM_D1'] = D_1
  Dados['NO_HOR_ODOM'] = Dados['NO_HOR_ODOM'].astype('float64')
  Dados['NO_HOR_ODOM_D1'] = Dados['NO_HOR_ODOM_D1'].astype('float64')
  Dados['Erro1'] = Dados['NO_HOR_ODOM_D1']-Dados['NO_HOR_ODOM']
  y = 1

Dados['Observação'] = ''
if y == 1:
  Dados.loc[((Dados['Erro1'])<=0) & (Dados['Observação']==''), 'Observação'] = "Erro 1: Horímetro apontado fora de sequência"
  # Dados.loc[((Dados['KM'])>=24) & (Dados['Observação']=='-'), 'Observação'] = "Erro 2: Horas trabalhadas maior que o limite diário"
  y=0

Result_HR = Dados[['INSTANCIA', 'PONTO', 'BOLETIM', 'FROTA', 'MARCA', 'MODELO', 'CATEGORIA', 'MATRICULA', 'OPERADOR', 'CARGO', 'HR_OPERACAO', 'MES', 'ANO', 'CD_MATERIAL', 'COMBUSTIVEL', 'OPER', 'OPERACAO', 'TIPO_RODADO', 'QTD', 'NO_HOR_ODOM', 'KM', 'Observação']]

########################||###########################

#Entrada da base
Dados2 = Dados1

#Pré-processamento (Filtro 'HR' | Conversão coluna KM e Unidade)
Dados2['KM'] = Dados2['KM'].astype('float64')
Dados2['QTD'] = Dados2['QTD'].astype('float64')
Dados2['FROTA'] = Dados2['FROTA'].astype('int64')
Dados2['TIPO_RODADO'] = Dados2['TIPO_RODADO'].astype('string')

#Filtrar equipamentos HR
Dados2 = Dados2[Dados2['TIPO_RODADO']=='KM']

#Tratamento da coluna data
lista_data = []
for i in Dados2['HR_OPERACAO']:
  i = str(i)
  convert = i.split()
  parameter = 'j'
  if int(len(convert))<2:
    lista_data.append(str(str(convert[0])+"  00:00:00"))
    parameter = 'i'
  if int(len(convert))>=2 and parameter=='j':
    lista_data.append(" ".join(str(i).split()))

Dados2['HR_OPERACAO'] = lista_data
Dados2['HR_OPERACAO']  = pd.to_datetime(Dados2['HR_OPERACAO'], format='%d/%m/%Y %H:%M:%S', infer_datetime_format=True)

#Ordenando os valores por FROTA e Horas
Dados2 = Dados2.sort_values(['FROTA', 'HR_OPERACAO']).reset_index(drop=True)

#Encontrando a diferença entre horímetro anterior e posterior
D_2 = list(Dados2['NO_HOR_ODOM'])
if len(D_2)>0:
  del D_2[0]
  D_2.append('0')

  Dados2['NO_HOR_ODOM_D1'] = D_2
  Dados2['NO_HOR_ODOM'] = Dados2['NO_HOR_ODOM'].astype('float64')
  Dados2['NO_HOR_ODOM_D1'] = Dados2['NO_HOR_ODOM_D1'].astype('float64')
  Dados2['Erro1'] = Dados2['NO_HOR_ODOM_D1']-Dados2['NO_HOR_ODOM']
  x = 1

Dados2['Observação'] = ''
if y == 1:
  Dados2.loc[((Dados2['Erro1'])<=0) & (Dados2['Observação']==''), 'Observação'] = "Erro 1: Horímetro apontado fora de sequência"
  # Dados2.loc[((Dados2['KM'])>=1000) & (Dados2['Observação']=='-'), 'Observação'] = "Erro 2: Horas trabalhadas maior que o limite diário estabelecido"
  y=0

Result_KM = Dados2[['INSTANCIA', 'PONTO', 'BOLETIM', 'FROTA', 'MARCA', 'MODELO', 'CATEGORIA', 'MATRICULA', 'OPERADOR', 'CARGO', 'HR_OPERACAO', 'MES', 'ANO', 'CD_MATERIAL', 'COMBUSTIVEL', 'OPER', 'OPERACAO', 'TIPO_RODADO', 'QTD', 'NO_HOR_ODOM', 'KM', 'Observação']]

#Unindo os dois dataframes resultantes
frames = [Result_HR, Result_KM]
result = pd.concat(frames)

# Ordenando os valores por FROTA e Horímetro
result = result.sort_values(['FROTA', 'HR_OPERACAO']).reset_index(drop=True)

#Removendo frotas avulsas e ferramentas manuais
values1=['LOCACAO AVULSA (KM)', 'LOCACAO AVULSA (HR)']
values2=['FERRAMENTAS MANUAIS']
values3=['BD-700','BDA-18.ORAE','B4T-706E', 'B4T-707 PA', 'BDA-22.0RAE', 'X14000', 'TG15000CXE3-XP', 'B4T-12000 E']
result = result[~result.CATEGORIA.isin(values1)]
result = result[~result.OPERACAO.isin(values2)]
result = result[~result.MODELO.isin(values3)]
result['HR_OPERACAO'] = result['HR_OPERACAO'].dt.strftime('%d/%m/%Y %H:%M:%S')


#Limpando observações desnecessárias na mudança de frota
lista_frotas = list(result['FROTA'])
del lista_frotas[0]
lista_frotas.append(0)
result['FROTA_VERIFICAR'] = lista_frotas
result.loc[((result['FROTA'])!=(result['FROTA_VERIFICAR'])) & (result['Observação']=="Erro 1: Horímetro apontado fora de sequência"), 'Observação'] = ""

#Selecionando as colunas
result = result[['INSTANCIA', 'PONTO', 'BOLETIM', 'FROTA', 'MARCA', 'MODELO', 'CATEGORIA', 'MATRICULA', 'OPERADOR', 'CARGO', 'HR_OPERACAO', 'MES', 'ANO', 'CD_MATERIAL', 'COMBUSTIVEL', 'OPER', 'OPERACAO', 'TIPO_RODADO', 'QTD', 'NO_HOR_ODOM', 'KM', 'Observação']]

# Salvar planilha Result
result.to_excel('/content/Correção_de_horímetros.xlsx', index = False)

- ## Verificação abastecimentos em outras unidades

In [ ]:
#Frotas que abastecem em outras unidades
frotas = list(Dados1['FROTA'].unique())
frotas_estranhas = []

for i in frotas:
  dados_frotas = Dados1[Dados1['FROTA']==i]
  if len(dados_frotas['INSTANCIA'].unique())>2:
    frotas_estranhas.append(i)
Divergentes = Dados1.loc[Dados1['FROTA'].isin(frotas_estranhas)]
Divergentes = Divergentes[['INSTANCIA', 'PONTO', 'BOLETIM', 'FROTA', 'MARCA', 'MODELO', 'CATEGORIA', 'MATRICULA', 'OPERADOR', 'CARGO', 'HR_OPERACAO', 'MES', 'ANO', 'CD_MATERIAL', 'COMBUSTIVEL', 'OPER', 'OPERACAO', 'TIPO_RODADO', 'QTD', 'NO_HOR_ODOM', 'KM']]
filter=['CARRO', 'LOCACAO AVULSA (KM)', 'LOCACAO AVULSA (HR)']
Divergentes = Divergentes[~Divergentes.CATEGORIA.isin(filter)]
if len(Divergentes)>0:
  Divergentes.to_excel('/content/Instancias_divergentes.xlsx', index = False)

- ## Abastecimentos duplicados

In [ ]:
#Abastecimentos duplicados
Dados123 = pd.read_excel('/content/Abast_07.xlsx')

Dados123['QTD'] = Dados123['QTD'].replace({',': '.'}, regex=True)
Dados123['NO_HOR_ODOM'] = Dados123['NO_HOR_ODOM'].replace({',': '.'}, regex=True)
Dados123['KM'] = Dados123['KM'].replace({',': '.'}, regex=True)

#Substituir deslocamentos vazios
Dados123['KM'] = Dados123['KM'].fillna(0)
Dados123['QTD'] = Dados123['QTD'].fillna(0)

#Pré-processamento (Filtro 'HR' | Conversão coluna KM e Unidade)
Dados123['KM'] = Dados123['KM'].astype('float64')
Dados123['QTD'] = Dados123['QTD'].astype('float64')
Dados123['FROTA'] = Dados123['FROTA'].astype('int64')
Dados123['TIPO_RODADO'] = Dados123['TIPO_RODADO'].astype('string')

# Dados123['HR_OPERACAO'] = Dados123['HR_OPERACAO'].dt.strftime("%d/%m/%Y %H")
duplicados = Dados123[Dados123.duplicated(subset=['FROTA', 'HR_OPERACAO', 'CD_MATERIAL', 'OPER', 'QTD'], keep=False)]
duplicados = duplicados.sort_values(['FROTA', 'HR_OPERACAO']).reset_index(drop=True)
if len(duplicados)>0:
  duplicados.to_excel('/content/Apontamentos_duplicados.xlsx', index = False)